In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM , Dropout , Dense

Using TensorFlow backend.
c:\users\rhyth\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\rhyth\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\rhyth\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [6]:
df_female = pd.read_csv('datasets/Project-gender/Female-Names.csv')
df_male = pd.read_csv('datasets/Project-gender/Male-Names.csv')

In [8]:
df_male.head()

,name,gender,race
0,barjraj,m,indian
1,ramdin verma,m,indian
2,sharat chandran,m,indian
3,birender mandal,m,indian
4,amit,m,indian


In [9]:
df = pd.concat([df_female , df_male])

In [10]:
char2idx = {}
char_set = [' ', '.','END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
index = 0
for ch in char_set:
    char2idx[ch] = index
    index += 1
   

In [11]:
def should_keep(word):
    if len(word) > 19:
        return False
    for ch in word:
        if ch not in set(char_set):
            return False
    return True

In [12]:
def clean(word):
    word = str(word)
    char_set = [' ', '.','END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
    name = word.lower()
    if should_keep(name):
        return name
    else:
        return None

In [13]:
df.name = df.name.apply(lambda x : clean(x))

In [14]:
df = df.dropna()

In [15]:
max_word_len = 20
def convert_one_hot(word):
    word_chars = []
    
    
    for ch in word:
        one_hot_vec = np.zeros(len(char_set),)
        one_hot_vec[char2idx[ch]] = 1
        word_chars.append(one_hot_vec)
    
    
    for _ in range(max_word_len-len(word)):
        one_hot_vec = np.zeros(len(char_set),)
        one_hot_vec[char2idx['END']] = 1
        word_chars.append(one_hot_vec)
    
    
    word_chars = np.array(word_chars)
    return word_chars

In [16]:
words = []
labels = []
for n , g in df.drop('race' , axis = 1).itertuples(index=False):
    words.append(convert_one_hot(n))
    labels.append(g)

In [17]:
words = np.array(words)
labels = np.array(labels)

In [18]:
words.shape

(27980, 20, 29)

In [19]:
ohe = OneHotEncoder()
y = ohe.fit_transform(labels.reshape(-1 , 1)).todense()

In [20]:
y[:10]

matrix([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])

In [21]:
model = Sequential()
model.add(LSTM(64 , input_shape = (20 , 29)  , return_sequences = True))
model.add(LSTM(64))
model.add(Dropout(.3))
model.add(Dense(2 , activation='sigmoid'))
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20, 64)            24064     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 57,218
Trainable params: 57,218
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(optimizer='rmsprop' , loss = 'binary_crossentropy' , metrics = ['acc'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [24]:
model = Sequential()
model.add(LSTM(128 , input_shape = (20 , 29)  , return_sequences = True))
model.add(LSTM(64))
model.add(Dropout(.3))
model.add(Dense(2 , activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 20, 128)           80896     
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 130,434
Trainable params: 130,434
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(optimizer='rmsprop' , loss = 'binary_crossentropy' , metrics = ['acc'])

In [27]:
model = Sequential()
model.add(LSTM(64 , input_shape = (20 , 29)  , return_sequences = True))
model.add(LSTM(64))
model.add(Dropout(.3))
model.add(Dense(2 , activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 20, 64)            24064     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 57,218
Trainable params: 57,218
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(optimizer='rmsprop' , loss = 'binary_crossentropy' , metrics = ['acc'])

In [29]:
model = Sequential()
model.add(LSTM(64 , input_shape = (20 , 29)  , return_sequences = True))
model.add(LSTM(64))
model.add(Dropout(.3))
model.add(Dense(2 , activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 20, 64)            24064     
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 130       
Total params: 57,218
Trainable params: 57,218
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(optimizer='rmsprop' , loss = 'binary_crossentropy' , metrics = ['acc'])

In [31]:
X_train, X_test, y_train, y_test = train_test_split(words, y, test_size=0.33, random_state=42)

In [32]:
model.fit(X_train , y_train , epochs = 10)

Epoch 1/10
18746/18746 [==============================] - 9s 476us/step - loss: 0.5330 - acc: 0.7340
Epoch 2/10
18746/18746 [==============================] - 8s 403us/step - loss: 0.3990 - acc: 0.8307
Epoch 3/10
18746/18746 [==============================] - 8s 405us/step - loss: 0.3408 - acc: 0.8647
Epoch 4/10
18746/18746 [==============================] - 8s 408us/step - loss: 0.3029 - acc: 0.8804
Epoch 5/10
18746/18746 [==============================] - 8s 416us/step - loss: 0.2752 - acc: 0.8960
Epoch 6/10
18746/18746 [==============================] - 9s 475us/step - loss: 0.2542 - acc: 0.9053
Epoch 7/10
18746/18746 [==============================] - 10s 521us/step - loss: 0.2399 - acc: 0.9145
Epoch 8/10
18746/18746 [==============================] - 10s 526us/step - loss: 0.2260 - acc: 0.9205
Epoch 9/10
18746/18746 [==============================] - 10s 552us/step - loss: 0.2151 - acc: 0.9252
Epoch 10/10
18746/18746 [==============================] - 10s 552us/step - loss: 0.2051

In [33]:
model.evaluate(X_test , y_test)

9234/9234 [==============================] - 1s 136us/step


[0.23385716327953215, 0.9234892787653465]

In [34]:
name = 'shreya'
model.predict(convert_one_hot(name).reshape(1 , 20 , 29))

array([[0.9720491 , 0.02914907]], dtype=float32)

In [35]:
name = 'anuj'
model.predict(convert_one_hot(name).reshape(1 , 20 , 29))

array([[0.05646877, 0.94643766]], dtype=float32)

In [36]:
name = 'rhythm'
model.predict(convert_one_hot(name).reshape(1 , 20 , 29))

array([[0.04145918, 0.964057  ]], dtype=float32)